<a href="https://colab.research.google.com/github/sherinjames-sj/GoogleColab_BigData/blob/main/ID142_CN7031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Big Data Analytics [CN7031] CRWK 2022-23**
1.   Sherin James U2298965
---


# **Initiate and Configure Spark**

---
PySpark is installed and the configuration of PySpark is prepared prior to its start-up.

In Spark, the SparkSession class is the entry point to all functions. SparkSession is also the entry point to Spark SQL, one of the very first objects that a Spark developer creates when developing a Spark SQL application. A Spark developer creates a SparkSession using the SparkSession class. As you use the builder method (that provides access to the Builder API, which you can use to configure the session), you will have access to the builder method (which gives you access to the Builder API).

To create an initial SparkSession, you will need to type the following commands.



In [ ]:
# PySpark is installed and the configuration of PySpark is prepared prior to its start-up

!pip3 install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 4.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=3b86348f443a9f87e2aa845259fc50c696b6c79ec3af7982fa2b5d821fb0c349
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Group id: 142") \
    .config("spark.some.config.option") \
    .getOrCreate()


# **Load Data**

---
Next, the Google Drive will be connected to Google Colaboratory and then the preferred dataset will be loaded. The attached folder contains all of the CSE-CIC-IDS2018 dataset which was created by the University of New Brunswick in order to analyze DDoS data. As you can see here we are attaching the entire folder of the dataset which consists of CSE-CIC-IDS2018. 
It is of utmost importance that we view the dataset within the collaborative environment. The cross reference process begins with loading the file correctly, ensuring that all the data is presented correctly, but also checking what the data is if we are loading it for the first time. By using this method, we find out what type of data the data is, how many columns and rows there are, how many entries there are, and what characteristics there are in the whole dataset.



In [ ]:
#mount your Google Drive:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Load the entire Dataset folder from google drive in colab:

groupid142_df = spark.read.load("/content/drive/MyDrive/DATASET", format="csv", inferSchema=True,  header=True)

In [ ]:
#Viewing the datafile:
groupid142_df.show()

In [ ]:
# Using printschema we recieve break down of the data along with the names of the columns and formats of the data, is presented

groupid142_df.printSchema()

In [ ]:
# As we move forward in this section, we will find out what types of data we will be working with
groupid142_df.dtypes

In [ ]:
#the count of total rows using the count method over data frame
groupid142_df.count()

In [ ]:
len (groupid142_df.columns)

In [ ]:
groupid142_df.columns

In [ ]:
groupid142_df.select('Label').groupBy('Label').count().orderBy('count', ascending=False).show()


# **Task 1: Spark SQL [30 marks]**

---



This Spark SQL module enables Hadoop Hive queries to be run up to 100 times faster on existing deployments and data using a programming abstraction called DataFrames. This is thanks to this software module. The Spark SQL module allows Hadoop Hive queries to run at up to 100 times faster on existing data. When it comes to performance, it can be viewed as being very similar to a distributed SQL query engine in the sense that it can perform in the same manner. As an additional feature, it also provides a type of abstraction which is known as DataFrames, which is similar in some ways to the SQL query engine in a database in terms of its functionality.

The following operations were used in this query:

SELECT: Returns a set of records as a result of its statement;
DISTINCT: It only returns distinct values (different);
FROM: Data values can be retrieved from a table or view;
WHERE: Records can be filtered with it;
GROUP BY: Creates summary rows based on rows with the same values;
LIMIT: Indicates how many records should be returned.

The aggregate functions which I used are:

COUNT: Returns how many rows there are in total;
MAX: Returns the highest value;
SUM: Returns the value calculated by this function;
AVG: The average of the values is returned by this function.


In [ ]:
from pyspark.sql.functions import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# The query below retrieves values from the view that I've created which consists of distinct values from DST PORT. 
# I have aggregated the columns `Subflow Fwd Byts`, `Tot Fwd Pkts`, `Flow Duration`, `Subflow Fwd Byts` and 
# then grouped them by DST PORT where the value of 
# the LABEL column was BENIGN and the data displayed in the view were from between a certain Timestamp. 

groupid142_df.createOrReplaceTempView("Subflow")

print("\n============================================= SUBFLOW =============================================\n")

sqlsparkDF= spark.sql("""SELECT Distinct `Dst Port` AS DESTINATION_PORT, 
count(`Subflow Fwd Byts`) AS COUNT_SUBFLOW_FWD_BYTS, max(`Tot Fwd Pkts`) AS MAX_TOTAL_FWD_PKTS, sum(`Flow Duration`) AS SUM_FLOW_DURATION, 
avg(`Subflow Fwd Byts`) AS AVG_SUBFLOW_FWD_BYTS FROM Subflow Group By `Dst Port`  Limit 10 """) 

sqlsparkDF.show()

# pandasDF = sqlsparkDF.toPandas()
# print("\n==================== BARH GRAPH ====================\n")
# pandasDF.plot(x = 'DESTINATION_PORT', y = 'MAX_TOTAL_FWD_PKTS', kind = 'barh')
# plt.xlabel("DESTINATION_PORT")
# plt.ylabel("MAX_TOTAL_FWD_PKTS")
# plt.show()

In [ ]:
# In the query below I have fetched for values from the view which I have created which consists of the values from Protocol, and 
# performed the aggregate functions on the column `Dst Port', `Fwd Pkts/s`, `Bwd Pkts/s`, `Fwd Header Len` and also performed the mathematical functions of ceiling and round and 
# then grouped them by Protocol where the value of the LABEL column was BENIGN.
# I have used 2 types of visualization method to plot my output and they are BAR graph and PIE graph.

groupid142_df.createOrReplaceTempView("PROTOCOL_LABEL")

sqlsparkDF= spark.sql("""SELECT Count('Dst Port') as COUNT_DESTINATION_PORT, Protocol as PROTOCOL, round(max(`Fwd Pkts/s`)) as ROUND_FWD_PKTS, 
ceiling(avg(`Bwd Pkts/s`)) as CEILING_BWD_PKTS, sum(`Fwd Header Len`) as SUM_FWD_HEADER_LEN from PROTOCOL_LABEL where Label='Benign' GROUP BY Protocol""")

# sqlsparkDF.show()

pandasDF = sqlsparkDF.toPandas()
print("\n==================== PIE GRAPH ====================\n")
pandasDF.plot(x = 'PROTOCOL', y = 'CEILING_BWD_PKTS', labels=['Protocol_6', 'Protocol_17', 'Protocol_0'], kind = 'pie')
#x is the color in the pie chart, i.e. PROTOCOL which is 6(y=4251), 17(y=1146), 0(y=145)
plt.legend(['4251_BWD_PKTS', '1146_BWD_PKTS', '145_BWD_PKTS'])
plt.show()



# **Task 2 - Part1: PySpark [45 marks]**

---


In [ ]:
# Analytical method 1: DESCRIPTIVE METHOD
#Here, the description of the whole dataset is displayed and represented visually as well.

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

datatrain = pd.read_csv('/content/drive/MyDrive/DATASET/02-14-2018.csv')
datatest = pd.read_csv('/content/drive/MyDrive/DATASET/02-14-2018.csv')
datatrain['source'] = 'train'
datatest['source'] = 'test'

# Concatinating both of the train and test data.
dataconcat = pd.concat([datatrain, datatest], ignore_index=True)
print(dataconcat)

# Test&Train Data
allcolumns = dataconcat.columns

# store numerical and categorical column in two different variables. It comes handy during visualizaion.
column_num = dataconcat._get_numeric_data().columns
column_cat = list(set(allcolumns)-set(column_num))

# below we are plotting the describe() function
# Pandas describe() function displays mean, stddev, percentile, count, and IQR values of the dataframe.
# By default describe() function only takes Int and float data type variables into consideration for calculations.

dataconcat.describe()

groupid142_describe = dataconcat.describe(include=['int64','float64'])
groupid142_describe.reset_index(inplace=True)

# To remove any variable from plot
groupid142_describe = groupid142_describe[groupid142_describe['index'] != 'count']

for i in column_num:
  if i in ['index']:
    continue
  sns.factorplot(x="index", y=i, data=groupid142_describe)
  plt.show()

In [ ]:
# Analytical method 2: CORRELATION METHOD

''' 
Correlation analysis is used to figure out if there exists a relationship between variables.
Correlation helps us to study both the strength and direction of the relationship between two sets of variables.
''' 

print("Correlation between the columns Dst Port and Bwd Pkt Len Std is : ")
result = groupid142_df.stat.corr("Dst Port", "Bwd Pkt Len Std")
print(result)

import seaborn as sns
# checking correlation using heatmap
#Loading dataset
flights = pd.read_csv('/content/drive/MyDrive/DATASET/02-14-2018.csv')
#plotting the heatmap for correlation
ax = plt.subplots(figsize=(10,9))
ax = sns.heatmap(flights.corr(), annot=True)

In [ ]:
# Analytical method 3: chisquare test gives us the pvalues and test satistics.

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import ChiSquareTest
VectAssem = VectorAssembler(
inputCols=["Fwd Act Data Pkts", "Fwd Seg Size Min"],
outputCol= "valuesofvector")
VectAssem
vect_DF = VectAssem.transform(groupid142_df).select("Fwd Act Data Pkts","valuesofvector")
vect_DF.show()

results = ChiSquareTest.test(vect_DF, "valuesofvector", "Fwd Act Data Pkts").head()

print("p-value = " + str (results.pValues))
print("test statistics =" + str (results.statistics))


# **Task 2 - Part2: PySpark [15 marks]**

---


# **Linear Regression**

 A Linear regression model is a statistical technique that is used to predict the value of one variable by comparing it with the value of another. An independent variable refers to the variable that you wish to predict, while a dependent variable refers to the variable you wish to use to forecast another variable.

With the use of linear regression, it is possible to determine the nature and strength of the relationship between a series of independent variables and a dependent variable. In order to develop models and make predictions, such as predicting the stock price of a company, this information can be used.




In [ ]:
# Machine Learning Technique: Linear Regression

from pandas import read_csv
#to import linear regression class
from sklearn.linear_model import LinearRegression
#to split data into training and test data
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.utils import class_weight

RANDOM_STATE_SEED = 12
data = read_csv("/content/drive/MyDrive/DATASET/02-14-2018.csv",  usecols=[0, 1, 3, 4, 5, 79])

#data.replace([np.inf, -np.inf], np.nan, inplace=True)

data.replace(to_replace=["FTP-BruteForce", "SSH-Bruteforce"], value="Malicious", inplace=True)

data["Label"].value_counts()

data1 = data[data["Label"] == "Benign"][:380943]
data2 = data[data["Label"] == "Malicious"][:380943]
data_equal = pd.concat([ data1,data2], axis =0)

data_equal.replace(to_replace="Benign", value=0, inplace=True)
data_equal.replace(to_replace="Malicious", value=1, inplace=True)


In [ ]:
data_equal.info()
train, test = train_test_split(data_equal, test_size=0.5)

In [ ]:
print("Full dataset:")
print("Benign: " + str(data_equal["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(data_equal["Label"].value_counts()[[1]].sum()))
print("\n---------------")

print("Training set:")
print("Benign: " + str(train["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(train["Label"].value_counts()[[1]].sum()))
print("\n---------------")

print("Test set:")
print("Benign: " + str(test["Label"].value_counts()[[0]].sum()))
print("Malicious: " + str(test["Label"].value_counts()[[1]].sum()))

In [ ]:
from scipy.sparse import random
#idex locator method of the df. This is the y. Our target variables.
labelpred = data_equal.iloc[:, 5]

#print(labelpred)

#Our input variables.
rem_col = data_equal.iloc[:, 0:4]

#print(rem_col)


x_train, x_test, y_train, y_test = train_test_split(rem_col, labelpred, random_state=0)
data['Label'].value_counts(normalize = True)


#create model

linreg = LinearRegression()

linreg.fit(x_train, y_train)

predictions = linreg.predict(x_test)

print(predictions, "\n")
print(y_test)

In [ ]:
accuracy_score(y_test, predictions.round())
